In [1]:
# Analysis of the train dataset
import pandas as pd
from IPython.display import display

train_path = "/Users/alexandre/Desktop/X/Python for Data Science/Projet Final Churn/train.parquet"

# Chargement des fichiers
train = pd.read_parquet(train_path)

# Train shape
print("Train shape:", train.shape)

# Aperçu de train
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
display(train.head())

# Colonnes de train sous forme tabulaire
display(pd.DataFrame(train.columns, columns=["Colonnes"]))

# Valeurs de page
display(train["page"].value_counts().head(20))

Train shape: (17499636, 19)


,status,gender,firstName,level,lastName,userId,ts,auth,page,sessionId,location,itemInSession,userAgent,method,length,song,artist,time,registration
0,200,M,Shlok,paid,Johnson,1749042,1538352001000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",278,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,524.32934,Ich mache einen Spiegel - Dream Part 4,Popol Vuh,2018-10-01 00:00:01,2018-08-08 13:22:21
992,200,M,Shlok,paid,Johnson,1749042,1538352525000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",279,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,178.02404,Monster (Album Version),Skillet,2018-10-01 00:08:45,2018-08-08 13:22:21
1360,200,M,Shlok,paid,Johnson,1749042,1538352703000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",280,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,232.61995,Seven Nation Army,The White Stripes,2018-10-01 00:11:43,2018-08-08 13:22:21
1825,200,M,Shlok,paid,Johnson,1749042,1538352935000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",281,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,265.50812,Under The Bridge (Album Version),Red Hot Chili Peppers,2018-10-01 00:15:35,2018-08-08 13:22:21
2366,200,M,Shlok,paid,Johnson,1749042,1538353200000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",282,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,471.69261,Circlesong 6,Bobby McFerrin,2018-10-01 00:20:00,2018-08-08 13:22:21


,Colonnes
0,status
1,gender
2,firstName
3,level
4,lastName
5,userId
6,ts
7,auth
8,page
9,sessionId


page
NextSong                     14291433
Thumbs Up                      789391
Home                           645259
Add to Playlist                409606
Roll Advert                    284837
Add Friend                     262147
Logout                         204700
Thumbs Down                    164964
Downgrade                      124248
Settings                       101191
Help                            89035
Upgrade                         37696
About                           33117
Save Settings                   20370
Error                           17294
Submit Upgrade                  11381
Submit Downgrade                 4425
Cancellation Confirmation        4271
Cancel                           4271
Name: count, dtype: int64

In [2]:
# Création de la target
cancellation_page = "Cancellation Confirmation"

cancellation_flag = (train["page"] == cancellation_page).astype(int)
user_target = (
    train.assign(target=cancellation_flag)
         .groupby("userId", as_index=False)["target"].max()
)

display(user_target.head())
display(user_target["target"].value_counts())


,userId,target
0,1000025,1
1,1000035,0
2,1000083,1
3,1000103,0
4,1000164,0


target
0    14869
1     4271
Name: count, dtype: int64

In [3]:
from functools import reduce

# 1) Conversion correcte du timestamp
train["ts"] = pd.to_datetime(train["ts"], unit="ms")
train["date"] = train["ts"].dt.date

# 3) Features par utilisateur

# 3.1 Nombre total d'événements
f_events = (
    train.groupby("userId")
         .size()
         .reset_index(name="n_events")
)

# 3.2 Nombre de jours actifs
f_days = (
    train.groupby("userId")["date"]
         .nunique()
         .reset_index(name="n_active_days")
)

# 3.3 Nombre de pages différentes
f_pages = (
    train.groupby("userId")["page"]
         .nunique()
         .reset_index(name="n_unique_pages")
)

# (Option sympa en plus) 3.4 Nombre de sessions
f_sessions = (
    train.groupby("userId")["sessionId"]
         .nunique()
         .reset_index(name="n_sessions")
)

# Fusion des features
feature_dfs = [f_events, f_days, f_pages, f_sessions]

user_features = reduce(
    lambda left, right: pd.merge(left, right, on="userId", how="left"),
    feature_dfs
)

# Dataset final
train_df = user_features.merge(user_target, on="userId", how="left")

display(train_df.head())
#Analyse du nombre de jours actifs
print(train_df.shape)
print(train_df["n_active_days"].describe())




,userId,n_events,n_active_days,n_unique_pages,n_sessions,target
0,1000025,2005,14,18,17,1
1,1000035,1556,21,16,21,0
2,1000083,596,9,15,11,1
3,1000103,75,3,10,3,0
4,1000164,1028,16,16,15,0


(19140, 6)
count    19140.000000
mean        10.745873
std          8.821739
min          1.000000
25%          4.000000
50%          8.000000
75%         15.000000
max         50.000000
Name: n_active_days, dtype: float64
